In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="TheBloke/WizardLM-13B-V1-1-SuperHOT-8K-GPTQ")

In [ ]:
#Settings:

cluster_labels_steps = cluster_labels_steps_kmeans
cluster_labels_results = cluster_labels_results_kmeans
num_clusters = kmeans_results.n_clusters

#-------

test_specs["stepsToReproduceClusters"] = [[] for _ in range(len(test_specs))]
test_specs["stepsToReproduceVector"] = [[0] * num_clusters for _ in range(len(test_specs))]

stepcounter = 0
testcounter = 0
for manualTest in test_specs["stepsToReproduceEmbeddings"]:
    for stepEmbedding in manualTest:
        if not isinstance(test_specs["stepsToReproduceClusters"].iloc[testcounter], list):
            test_specs["stepsToReproduceClusters"].iloc[testcounter] = []
        test_specs["stepsToReproduceClusters"].iloc[testcounter].append(
            cluster_labels_steps[stepcounter]
        )
        test_specs["stepsToReproduceVector"].iloc[testcounter][
            cluster_labels_steps[stepcounter]
        ] += 1
        stepcounter += 1
    testcounter += 1

test_specs["expectedResultsClusters"] = [[] for _ in range(len(test_specs))]
test_specs["expectedResultsVector"] = [[0] * num_clusters for _ in range(len(test_specs))]


resultcounter = 0
testcounter = 0
for manualTest in test_specs["expectedResultsEmbeddings"]:
    for stepEmbedding in manualTest:
        if not isinstance(test_specs["expectedResultsClusters"].iloc[testcounter], list):
            test_specs["expectedResultsClusters"].iloc[testcounter] = []
        test_specs["expectedResultsClusters"].iloc[testcounter].append(
            cluster_labels_results[resultcounter]
        )
        test_specs["expectedResultsVector"].iloc[testcounter][
            cluster_labels_results[resultcounter]
        ] += 1
        resultcounter += 1
    testcounter += 1

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/WizardLM-13B-V1-1-SuperHOT-8K-GPTQ"
model_basename = "wizardlm-13b-v1.1-superhot-8k-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    device_map="auto",
    use_triton=use_triton,
    quantize_config=None,
)

model.seqlen = 8192

# Note: check the prompt template is correct for this model.
prompt = "Tell me about AI"
prompt_template = f"""USER: {prompt}
ASSISTANT:"""

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)

print(pipe(prompt_template)[0]["generated_text"])

In [ ]:
from transformers import pipeline, logging

logging.set_verbosity(logging.DEBUG)

# Initialize a text-generation pipeline
generator = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta")

# Generate text based on a prompt
prompt = "In a distant future, humanity has"
generated_text = generator(
    prompt, max_length=50, num_return_sequences=5, temperature=0.7
)

for i, text in enumerate(generated_text):
    print(f"Generated text {i+1}:")
    print(text["generated_text"])

In [ ]:
from transformers import pipeline
import torch

# Create a pipeline for the BERT model
bert_pipeline = pipeline(
    "feature-extraction",
    model="bert-base-uncased",
    device=0 if torch.cuda.is_available() else -1,
)


def ensure_string_format(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, list):
        return [
            str(item) for item in data
        ]  # Convert each item to string if not already
    else:
        print("WARNING, WRONG DATA")
        print(data)
        return str(data)  # Convert other types to string


def get_bert_embedding(text):
    # Get embeddings using the pipeline
    text = ensure_string_format(text)
    embedding = bert_pipeline(
        text, return_tensors=True, padding=True, truncation=True, max_length=128
    )
    sentence_embedding = torch.mean(torch.tensor(embedding), dim=1).numpy()
    # print(embedding[0])
    # Average the token embeddings
    return sentence_embedding


# TODO: Try to use parallel embedding feature from pipelines by giving it the entire list at once
def get_bert_embedding_for_list(list_of_text):
    return [get_bert_embedding(text) for text in list_of_text]


# Embeddings for each line
test_info["Category_Embedding"] = test_info["Category"].apply(get_bert_embedding)
test_info["TestName_Embedding"] = test_info["Test Name"].apply(get_bert_embedding)
test_specs["expectedResultEmbeddings"] = test_specs["expectedResult"].apply(
    get_bert_embedding_for_list
)
test_specs["stepsToReproduceEmbeddings"] = test_specs["stepsToReproduce"].apply(
    get_bert_embedding_for_list
)

/tmp/ipykernel_9891/2074761894.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_embedding = torch.mean(torch.tensor(embedding), dim=1).numpy()
/home/fedrive/Studium/Semester_5/Seminar/NLP-Test-Prioritization/python_src/.venv/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/fedrive/Studium/Semester_5/Seminar/NLP-Test-Prioritization/python_src/.venv/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/tmp/ipykernel_9891/2074761894.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTen

KeyboardInterrupt: 